In [2]:
import itertools
import torch
import nltk
import re
import wget
import sys
import os
import pandas as pd
import numpy as np
import random

from collections import defaultdict, Counter

nltk.download('punkt', quiet=True) # this module is used to tokenize the text

SEED = 1234
random.seed(SEED)

In [3]:
# Some utilities to manipulate the corpus

def preprocess(text):
    """Strips #comments and empty lines from a string
    """
    result = []
    for line in text.split("\n"):
        line = line.strip()
        line = re.sub('#.*$', '', line)
        if line != '':
            result.append(line)
    return result

def nltk_normpunc_tokenize(str):
    return nltk.tokenize.word_tokenize(str.lower())

def split(list, portions, offset):
    return ([list[i] for i in range(0, len(list)) if i%portions != offset],
          [list[i] for i in range(0, len(list)) if i%portions == offset])

def SMSSpamCollection_tokenize(lines):
    result = []
    for line in lines:
        # tokenize
        tokens = nltk_normpunc_tokenize(line)
        if tokens[0] == "ham":
            tokens[0] = "HAM:"
        elif tokens[0] == "spam":
            tokens[0] = "SPAM:"
        else:
            raise ValueError("format problem - uncategorized SMS")
        # add a start of message token
        result += ["<s>"] + tokens

    return result

def postprocess(tokens):
    return ' '.join(tokens)\
                .replace("<s> ", "\n")

Download the corpus

In [7]:
corpus_filename = ("https://github.com/DanielLevi6/NLP-home-assignment/tree/main/data/"
                  "SMSSpamCollection.txt")
os.makedirs('data', exist_ok=True)
wget.download(corpus_filename, out="data/")

-1 / unknown

ValueError: not enough values to unpack (expected 2, got 1)

Load the corpus

In [ ]:
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize, word_tokenize

lines = preprocess(gutenberg.raw('carroll-alice.txt'))
train_lines, test_lines = split(lines, 12, 0)

train_tokens = tokenize_lines(train_lines)
test_tokens = tokenize_lines(test_lines)

In [ ]:
# Extract vocabulary from dataset
vocabulary = list(set(train_tokens))

In [ ]:
# Creating the n-grams
def all_ngrams(vocabulary, n):
    return list(itertools.product(vocabulary, repeat=n))

def ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(0, len(tokens)-n+1)]

In [ ]:
print(train_tokens[:6])
print(ngrams(train_tokens[:6], 3))

In [ ]:
# Counting the ngrams
def ngram_counts(vocabulary, tokens, n):
    context_dict = defaultdict(lambda: defaultdict(int))
    for context in all_ngrams(vocabulary, n-1):
        for target in vocabulary:
            context_dict[context][target] = 0

    for ngram, count in Counter(ngrams(tokens, n)).items():
        context_dict[ngram[:-1]][ngram[-1]] = count

    return context_dict

In [ ]:
unigram_counts = ngram_counts(vocabulary, train_tokens, 1)
bigram_counts = ngram_counts(vocabulary, train_tokens, 2)
trigram_counts = ngram_counts(vocabulary, train_tokens, 3)